In [2]:
# Modulos básicos
import numpy as np
import time
#from pylab import imshow
import matplotlib.pyplot as plt
from tqdm import tqdm, tnrange, tqdm_notebook
# Modulo para manejo de fecha
from datetime import datetime, timedelta
# Modulos para astrofisica/solar
import astropy
from sunpy.net import vso
import astropy.units as u
from sunpy.map import Map
from astropy.io import fits # to fix headers
# Custom-made methods and classes for fixing headers
#from lib.CompatMaps import sinehpc_wcs_frame_mapping

# Obtener los datos
Se usa el cliente VSO de SunPy para obtener los datos automáticamente, solo se tienen que cambiar las fechas correspondientes. Fechas de interés para el proyecto son:
* 2012/01/29 a 2012/01/30
* 2013/03/04 a 2013/03/09
* 2014/09/23 a 2014/09/28
* 2015/09/03 a 2015/09/08
* 2016/03/11 a 2016/03/16
* 2016/08/28 a 2016/08/31
* 2016/06/13 a 2016/06/16
* 2016/03/29 a 2016/04/01
* 2016/01/29 a 2016/02/01
* 2016/08/13 a 2016/08/16
* 2012/11/01 - 2012/11/06
* 2015-11-25 - 2015-11-27
* 2012/02/28 - 2012/03/02
* 2012/02/18 - 2012/02/21
* 2011/09/29 - 2011/10/02
* 2011/10/08 - 2011/10/11
* 2012/05/01 - 2012/05/04
* 2012/07/01 - 2012/07/04

In [5]:
# defining datetime range and number of samples 
dates = []  # where the dates pairs are going to be stored
date_start = datetime(2012,7,1,0,0,0)
date_end = datetime(2012,7,3,23,59,59)
date_samples = 35  # Number of samples to take between dates
date_delta = (date_end - date_start)/date_samples  # How frequent to take a sample
date_window = timedelta(minutes=1.0)
temp_date = date_start
while temp_date < date_end:
    dates.append((str(temp_date),str(temp_date+date_window)))
    temp_date += date_delta 

In [6]:
for i in range(3):
    # definir instrumento
    instrument = 'hmi'
    # definir rango de longitud de onda (min,max)
    #wavelength = 400*u.nm , 700*u.nm

    # Query data - Buscar datos en esas fechas
    t = 0
    for i in dates:
        tstart, tend = i[0], i[1]
        #data_client = vso.VSOClient()
        data_client = vso.VSOClient(url='https://vso.nascom.nasa.gov/API/VSOi_rpc_literal.wsdl') # workaround when VSO server fail 
        # more info at: https://riot.im/app/#/room/!MeRdFpEonLoCwhoHeT:matrix.org/$14939136771403280rCVLc:matrix.org 
        data_query = data_client.query(vso.attrs.Time(tstart, tend), \
                                       vso.attrs.Instrument(instrument), vso.attrs.Physobs("intensity"))
        print("Found ",len(data_query)," records from ", tstart, " to ", tend)
        print("Time range: ", data_query.time_range())
        print("Size in KB: ", data_query.total_size())
        data_dir = '/home/ivan/projects/Physics/solar/solar-physics-ex/rotation/data/set18/{file}.fits'
        results = data_client.get(data_query, path=data_dir)
        if t%2 == 0:
            time.sleep(30)
        t+=1

Found  1  records from  2012-07-01 00:00:00  to  2012-07-01 00:01:00
Time range:  (datetime.datetime(2012, 7, 1, 0, 0, 40), datetime.datetime(2012, 7, 1, 0, 0, 41))
Size in KB:  0
Found  2  records from  2012-07-01 02:03:25.685714  to  2012-07-01 02:04:25.685714
Time range:  (datetime.datetime(2012, 7, 1, 2, 3, 40), datetime.datetime(2012, 7, 1, 2, 4, 26))
Size in KB:  0
Found  1  records from  2012-07-01 04:06:51.371428  to  2012-07-01 04:07:51.371428
Time range:  (datetime.datetime(2012, 7, 1, 4, 7, 25), datetime.datetime(2012, 7, 1, 4, 7, 26))
Size in KB:  0
Found  2  records from  2012-07-01 06:10:17.057142  to  2012-07-01 06:11:17.057142
Time range:  (datetime.datetime(2012, 7, 1, 6, 10, 25), datetime.datetime(2012, 7, 1, 6, 11, 11))
Size in KB:  0
Found  1  records from  2012-07-01 08:13:42.742856  to  2012-07-01 08:14:42.742856
Time range:  (datetime.datetime(2012, 7, 1, 8, 14, 10), datetime.datetime(2012, 7, 1, 8, 14, 11))
Size in KB:  0
Found  2  records from  2012-07-01 10:17

## Acquiring data from helioviewer
To this date (04/05/2017) the VSO server for fits files docs.virtualsolar.org is down and has been for some hours. So I had to choose to use Helioviewer to download data, which come in jpg/png files.

In [ ]:
from sunpy.net.helioviewer import HelioviewerClient

hv = HelioviewerClient()
datasources = hv.get_data_sources()

# print a list of datasources and their associated ids
for observatory, instruments in datasources.items():
    for inst, detectors in instruments.items():
        for det, measurements in detectors.items():
            for meas, params in measurements.items():
                print("%s %s: %d" % (observatory, params['nickname'], params['sourceId']))

In [ ]:
filepath = hv.download_jp2('2012/07/05 00:30:00', directory="data/HMI/set1", observatory='SDO', instrument='HMI', detector='HMI', measurement='continuum')
hmi = Map(filepath)
#xrange = Quantity([200, 550], 'arcsec')
#yrange = Quantity([-400, 200], 'arcsec')
hmi.submap(xrange, yrange).peek()
# Si falla
# < Cadair> mefistofeles: install glymur and openjpeg >1.5